<a href="https://colab.research.google.com/github/littlebiggeo/csdms-clinic-2020/blob/master/notebooks_anuga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSDMS HRT Data-Model Integration Workshop

## Use of ANUGA in Flood Modeling

Modeling flooding in the San Luis Mountains 


## Setup the Environment

Run the following cell to install the dependencies and some extra code for visualising on Colaboratory.

Wait until you see the comment *(5) Ready to go* before proceeding to subsequent commands. 

In [0]:
try:
  # On colab we can install all the packages we need via the notebook
  #
  # First download the clinic repository
  import os
  os.chdir('/content')
  !git clone https://github.com/littlebiggeo/csdms-clinic-2020.git

  # Now install environment using tool
  !/bin/bash /content/csdms-clinic-2020/anuga/install_anuga.sh
 
except:
  pass

if not 'workbookDir' in globals():
    workbookDir = os.getcwd()

import sys
sys.path.append(os.path.join(workbookDir,"csdms-clinic-2020"))

## Setup Inline graphics and import anuga

In [0]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Allow inline jshtml animations
from matplotlib import rc
rc('animation', html='jshtml')

## Import ANUGA

In [0]:
import anuga

## Read in Data

We have included some topography data and extent data in our `csdms-clinic-2020` download. 

Let's read that in and create a mesh associated with it. 

In [0]:
data_dir = '/content/csdms-clinic-2020/data/sanluis'

# Polygon defining broad area of interest
bounding_polygon = anuga.read_polygon(os.path.join(data_dir,'extent.csv'))


# Polygon defining particular area of interest
channel_polygon = anuga.read_polygon(os.path.join(data_dir,'channel.csv'))


# Elevation Data
topography_file = os.path.join(data_dir,'dem1.asc')


# Resolution for most of the mesh
base_resolution = 100.0  # m^2

# Resolution in particular area of interest 
channel_resolution = 25.0 # m^2

interior_regions = [[channel_polygon, channel_resolution]]

## Create and View Domain


Note that we use a base_resolution to ensure a reasonable refinement overthe whole region, and we use interior_regions to refine the mesh in the area of interest. In this case we pass a list of polygon, resolution pairs. 

In [0]:
domain = anuga.create_domain_from_regions(
            bounding_polygon,
            boundary_tags={'bottom': [0],
                           'right':  [1],
                           'top':    [2],
                           'left':   [3]},
            maximum_triangle_area=base_resolution,
            interior_regions=interior_regions)


domain.set_name('sanluis') # Name of sww file
dplotter = anuga.Domain_plotter(domain)  
plt.triplot(dplotter.triang, linewidth = 0.1);

## Setup Initial Conditions

We must choose friction and stage values and populate our mesh with elevation. In this case we will do this using the DEM file `dem1.asc` .



In [0]:
domain.set_quantity('elevation', filename=topography_file, location='centroids') # Use function for elevation
domain.set_quantity('friction', 0.015, location='centroids')                        # Constant friction 
domain.set_quantity('stage', expression='elevation', location='centroids')         # Dry Bed 

plt.tripcolor(dplotter.triang, 
              facecolors = dplotter.elev, 
              cmap='Greys_r')
plt.colorbar();
plt.title("Elevation");

## Setup Boundary Conditions

The rectangular domain has 4 tagged boundaries, left, top, right and bottom. We need to set boundary conditons for each of these tagged boundaries. We can set Transmissive type BC on the outflow boundaries and reflective on the others. 

In [0]:
Br = anuga.Reflective_boundary(domain)
Bt = anuga.Transmissive_boundary(domain)

domain.set_boundary({'bottom':   Br,
                     'right':    Bt, # outflow
                     'top':      Br, # outflow
                     'left':     Bt})

## Set discharge operator

We specify a constant discharge condition at the upstream boundary

In [0]:
# Setup discharge inlet
center = (448046.0, 4055577.0)
radius = 5.0
region0 = anuga.Region(domain, center=center, radius=radius)
fixed_inflow = anuga.Inlet_operator(domain, region0 , Q=200)

## Run the Evolution

We evolve using a `for` statement, which evolves the quantities using the shallow water wave solver. The calculation `yields` every `yieldstep` seconds, up to a given `duration`.

In [0]:
for t in domain.evolve(yieldstep=5, duration=200):
  
    #dplotter.plot_depth_frame()
    dplotter.save_depth_frame(vmin=0.0, vmax=3.0)
    
    domain.print_timestepping_statistics()

    
# Read in the png files stored during the evolve loop
dplotter.make_depth_animation() 

## SWW File

Plot maximum depth and speed at the last time slice of the .sww file

In [0]:
# Create a wrapper for contents of sww file
swwfile = 'sanluis.sww'
splotter = anuga.SWW_plotter(swwfile)


# Plot Depth and Speed at the last time slice
plt.subplot(121)
splotter.triang.set_mask(None)
plt.tripcolor(splotter.triang, 
              facecolors = splotter.depth[-1,:], 
              cmap='viridis')

plt.title("Depth")


plt.subplot(122)
splotter.triang.set_mask(None)
plt.tripcolor(splotter.triang, 
              facecolors = splotter.speed[-1,:], 
              cmap='viridis')

plt.title("Speed");


### Monitor stage at chosen locations

The data file ObservationPoints.csv contains synthetic gauge locations that we can use to query flow characteristics throughout the model

In [0]:
point_observations = np.genfromtxt(
    os.path.join(data_dir,'ObservationPoints.csv'),
    delimiter=",",skip_header=1)

# Convert to absolute corrdinates
xc = splotter.xc + splotter.xllcorner
yc = splotter.yc + splotter.yllcorner

nearest_points = []
for row in point_observations:
    nearest_points.append(np.argmin( (xc-row[0])**2 + (yc-row[1])**2 ))
 
loc_id = point_observations[:,2]

fig, ax = plt.subplots()
ax.plot(loc_id, splotter.stage[-1,nearest_points], '*g', label='Anuga')

plt.xticks(range(0,5))
plt.xlabel('ID')
plt.ylabel('Stage')
ax.legend()

plt.show()


### Run model again with differing friction


In [0]:
# Resolution for most of the mesh
base_resolution = 200.0  # m^2

# Resolution in particular area of interest 
channel_resolution = 25.0 # m^2

domain = anuga.create_domain_from_regions(
            bounding_polygon,
            boundary_tags={'bottom': [0],
                           'right':  [1],
                           'top':    [2],
                           'left':   [3]},
            maximum_triangle_area=base_resolution,)


domain.set_name('sanluis2') # Name of sww file
domain.set_low_froude(1)

# Setup Initial Conditions
domain.set_quantity('elevation', filename=topography_file, location='centroids') # Use function for elevation
domain.set_quantity('friction', 0.02, location='centroids')                        # Constant friction 
domain.set_quantity('stage', expression='elevation', location='centroids')         # Dry Bed 

# Setup BC
Br = anuga.Reflective_boundary(domain)
Bt = anuga.Transmissive_boundary(domain)


# NOTE: We need to assign a BC to the interior boundary region.
domain.set_boundary({'bottom':   Br,
                     'right':    Bt, # outflow
                     'top':      Br, 
                     'left':     Bt, # outflow
                     'interior': Br})


# Setup inlet flow
center = (448046.0, 4055577.0)
radius = 5.0
region0 = anuga.Region(domain, center=center, radius=radius)
fixed_inflow = anuga.Inlet_operator(domain, region0 , Q=200)


dplotter = anuga.Domain_plotter(domain)  
plt.triplot(dplotter.triang, linewidth = 0.1);

### Evolve

In [0]:
for t in domain.evolve(yieldstep=5, duration=200):
  
    #dplotter.plot_depth_frame()
    dplotter.save_depth_frame(vmin=0.0, vmax=3)
    
    domain.print_timestepping_statistics()

    
# Read in the png files stored during the evolve loop
dplotter.make_depth_animation() 

### Read in SWW File and Compare
 

In [0]:
# Create a wrapper for contents of sww file
swwfile2 = 'sanluis2.sww'
splotter2 = anuga.SWW_plotter(swwfile2)

# Convert to absolute corrdinates
xc = splotter2.xc + splotter2.xllcorner
yc = splotter2.yc + splotter2.yllcorner

nearest_points_2 = []
for row in point_observations:
    nearest_points_2.append(np.argmin( (xc-row[0])**2 + (yc-row[1])**2 ))
 
loc_id = point_observations[:,2]

fig, ax = plt.subplots()
ax.plot(loc_id, splotter2.stage[-1,nearest_points_2], '*g', label='Anuga1')
ax.plot(loc_id, splotter.stage[-1,nearest_points], '*k', label='Anuga0')


plt.xticks(range(0,5))
plt.xlabel('ID')
plt.ylabel('Stage')
ax.legend()

plt.show()

[Next Notebook](https://colab.research.google.com/github/stoiver/anuga-clinic-2018/blob/master/notebooks/notebook3.ipynb)